In [29]:
# pip install pandas openpyxl sentence-transformers faiss-cpu

In [30]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import faiss

In [31]:
# Step 1: Load Excel file into DataFrame
df = pd.read_excel("sample_it_tickets.xlsx")  

# Step 2: Convert each row to a string with headers
texts = []
for _, row in df.iterrows():
    row_str = ' | '.join([f"{col}: {row[col]}" for col in df.columns])
    texts.append(row_str)

texts

['Ticket ID: T001 | Issue Type: Network | Description: Internet connectivity issue | Priority: High | Status: Resolved | Resolution Time (hrs): 2.0 | Assigned To: Alice',
 'Ticket ID: T002 | Issue Type: Hardware | Description: Laptop not booting | Priority: Medium | Status: Resolved | Resolution Time (hrs): 5.0 | Assigned To: Bob',
 'Ticket ID: T003 | Issue Type: Software | Description: Application crash on startup | Priority: High | Status: In Progress | Resolution Time (hrs): 3.0 | Assigned To: Charlie',
 'Ticket ID: T004 | Issue Type: Access | Description: Unable to access shared drive | Priority: Low | Status: Resolved | Resolution Time (hrs): 1.0 | Assigned To: David',
 'Ticket ID: T005 | Issue Type: Email | Description: Email not syncing | Priority: Medium | Status: Resolved | Resolution Time (hrs): 2.0 | Assigned To: Eve',
 'Ticket ID: T006 | Issue Type: Network | Description: VPN connection drops frequently | Priority: High | Status: Open | Resolution Time (hrs): nan | Assigned

In [32]:
# Embed the text
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(texts)

# Create FAISS index
dimension = embeddings[0].shape[0]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

/workspaces/Gen-AI-Learning/.venv/lib/python3.12/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [33]:
DISTANCE_THRESHOLD = 1.5 # more than 1.5 is not good match
def search_similar(query, top_k=5):
    query_embedding = model.encode([query])
    distances, indices = index.search(query_embedding, top_k)
    results = []
    for distance, indx in zip(distances[0], indices[0]):
        if distance <= DISTANCE_THRESHOLD:
            results.append(texts[indx])
    return results

In [34]:
query = "What are the network issues?"
results = search_similar(query, top_k=3)
results

['Ticket ID: T001 | Issue Type: Network | Description: Internet connectivity issue | Priority: High | Status: Resolved | Resolution Time (hrs): 2.0 | Assigned To: Alice',
 'Ticket ID: T006 | Issue Type: Network | Description: VPN connection drops frequently | Priority: High | Status: Open | Resolution Time (hrs): nan | Assigned To: Frank']